# Bert For Summarization (tiếp tục)
Trong bài thực hành về nhà này ta sẽ tiếp tục sử dụng mã nguồn của phân thực hành trên lớp, tuy nhiên ta sẽ tự làm phần xử lý dữ liệu với bộ dataset CNN_stories. 

Download dữ liệu theo đường dẫn sau : https://drive.google.com/file/d/1PJ0gyoqflubbdC8jJgQnqu3kbu4cyHCz/view?usp=sharing
Download Stanford CoreNLP theo đường dẫn sau : https://drive.google.com/file/d/1PWJMi8jDTVOAvxA0kv9AyNKglhVlQtp4/view?usp=sharing

## Bước 1: Cài đặt môi trường và thư viện

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14
!mkdir bai14_home
%cd /content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14/bai14_home

/content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14
/content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14/bai14_home


Mã nguồn gốc của bài thực hành [tại đây](https://github.com/nlpyang/BertSum.git). Mã nguồn mà ta sử dụng đã fix một vài lỗi nhỏ để chạy được

Ta lấy bộ dataset CNN stories cho bài thực hành [tại đây](https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ)

Clone mã nguồn gốc

In [4]:
!git clone https://github.com/nlpyang/BertSum.git
%cd /content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14/BertSum

fatal: destination path 'BertSum' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14/BertSum'
/content/drive/MyDrive/NLP/Đáp án cho bài tập về nhà/Lesson14/bai14_home


Cài đặt các thư viện cần thiết

In [ ]:
!pip install torch==1.1.0 torchvision==0.3.0
!pip install pytorch_pretrained_bert
!pip install pyrouge
!pip install tensorboardX

Cài đặt thư viện CoreNLP

In [ ]:
# Lưu ý : chuyển file zip stanford-corenlp-latest.zip vào folder BertSum trước khi thực hiện code này
!unzip stanford-corenlp-latest.zip
!ls

Cài đặt Java JDK để chạy thư viện Core-NLP

In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [ ]:
corenlp_dir = "/content/drive/MyDrive/bai14_home/BertSum/stanford-corenlp-4.4.0/stanford-corenlp-4.4.0.jar"
import os
os.environ["CLASSPATH"] = corenlp_dir

## Bước 2: Xử lý dữ liệu

### 2.1. Giải nén dữ liệu

In [ ]:
!mkdir /content/drive/MyDrive/bai14_home/BertSum/raw_stories
!tar -xvzf cnn_stories.tgz -C/content/drive/MyDrive/bai14_home/BertSum/raw_stories

In [ ]:
!mkdir -p /content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized
!touch /content/drive/MyDrive/bai14_home/BertSum/logs/cnndm.log

In [ ]:
import os
os.chdir("/content/drive/MyDrive/bai14_home/BertSum")

### 2.2. Hàm tokenize để tách từ và tách câu trong các bản ghi.

Hướng dẫn: Ta sử dụng câu lệnh như sau:
python preprocess.py -mode tokenize -raw_path RAW_PATH -save_path TOKENIZED_PATH
- `<RAW_PATH>`: Folder chứa các file cần xử lý của bước này (cần được tách từ)
- `<TOKENIZED_PATH>`: Folder chứa các file đã được tách từ

In [ ]:
!python src/preprocess.py -mode tokenize \
-raw_path /raw_stories/cnn/stories \
-save_path merged_stories_tokenized

### 2.3. Format các file json vừa được xử lý về dạng json đơn giản hơn

**Hướng dẫn**: Chạy câu lệnh: \
 `python src/preprocess.py -mode format_to_lines 
-raw_path <RAW_PATH> \
-save_path <JSON_DATA> -map_path <MAP_PATH> \
-lower `

Trong đó:
- `<RAW_PATH>`: Folder chứa các file cần xử lý của bước này (đầu ra của bước trước)
- `<JSON_DATA>`: Folder chứa các file kết quả của bước này
-  `<MAP_PATH>`: Folder chứa các file urls.


In [ ]:
# YOUR CODE HERE
!python src/preprocess.py -mode format_to_lines \
-raw_path merged_stories_tokenized \
-save_path json_data \
-map_path urls \
-lower 
# YOUR CODE HERE

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/5bf540c3df15a780eb231af6c1b6263eec08139f.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/8fbb90165c06e353905dd96c9465e9b5f65d8088.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/dfd01235f7d2f00d5b7051059374c6e2968bea64.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/cb58564cf4782a5736d4e411b206b801be6a48f9.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/14ad25b8836536c9a83a610182972b7aefaf8ae0.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/3b13a7fecb77be63cac1e9286b83f61b5784f387.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/71cb719d5f47330d97a5ccb8acdc94db8a81084d.story.json
/content/drive/MyDrive/bai14_home/BertSum/merged_stories_tokenized/6935e09175390c03b40d06280fe7c01d2f1629c0.story.js

### 2.4. Format các file json vừa chạy về định dạng `.pt`
**Hướng dẫn**: Chạy câu lệnh sau: \
`!python src/preprocess.py -mode format_to_bert \
-raw_path <JSON_DATA> -save_path <BERT_DATA> -oracle_mode greedy \
-n_cpus 4 -log_file <LOG_FILE>`

Trong đó:
- `<JSON_DATA>`: Folder chứa các file json kết quả xử lý của bước trước
- `<BERT_DATA>`: Folder chứa file kết quả xử lý của bước này
- `<LOG_FILE>`: File ghi lại log của quá trình chạy

In [ ]:
# YOUR CODE HERE
!python src/preprocess.py -mode format_to_bert -raw_path merged_stories_tokenized -save_path bert_data -oracle_mode greedy -n_cpus 4 -log_file logs/preprocess.log
# YOUR CODE HERE

[]
[]
[]


Quan sát dữ liệu

In [ ]:
import torch

def get_data(subset):
  assert subset in ["train", "test", "valid"]
  data_path = "bert_data"
  data = []
  for file in os.listdir(data_path):
    if subset in file:
      data_from_file = torch.load(os.path.join(data_path, file))
      data.extend(data_from_file)
  return data

In [ ]:
train_data = get_data("train")

In [ ]:
print(f"Training data có số mẫu dữ liệu như sau: {len(train_data)}")
print(train_data[0].keys())
print("src:", train_data[0]["src"])
print("labels:", train_data[0]["labels"])
print("segs:", train_data[0]["segs"])
print("clss:", train_data[0]["clss"])
print("src_txt:", train_data[0]["src_txt"])
num_wrd_src = 0
for sent in train_data[0]["src_txt"]:
  num_wrd_src += len(sent.split())
print("Độ dài src_txt: ", num_wrd_src)
print("tgt_txt:", train_data[0]["tgt_txt"])
print("Độ dài tgt_txt: ", len(train_data[0]["tgt_txt"].split(" ")))

## Phần 3: Huấn luyện mô hình và đánh giá

### 3.1. Huấn luyện mô hình

In [ ]:
# Tạo các file và folder cần thiết
!mkdir -p /content/drive/MyDrive/bai14_home/BertSum/checkpoints
!touch /content/drive/MyDrive/bai14_home/BertSum/logs/results.log

In [ ]:
# Kiểm tra thiết bị
import torch
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [ ]:
# Tạo thư mục chứa file model
!mkdir -p /content/drive/MyDrive/bai14_home/BertSum/checkpoints/classifier
!mkdir -p  /content/drive/MyDrive/bai14_home/BertSum/checkpoints/transformer
!mkdir -p /content/drive/MyDrive/bai14_home/BertSum/checkpoints/rnn

In [ ]:
!python src/train.py -mode train -encoder classifier -dropout 0.1 \
-bert_data_path bert_data/ \
-model_path checkpoints/classifier \
-gpu_ranks 0 -visible_gpus 0   \
-lr 2e-3 \
-report_every 100 \
-save_checkpoint_steps 100 \
-batch_size 3000 \
-decay_method noam \
-train_steps 500 \
-accum_count 2 \
-log_file /logs/results.log \
-use_interval true -warmup_steps 1000

[2020-09-29 03:23:53,195 INFO] Device ID 0
[2020-09-29 03:23:53,195 INFO] Device cuda
[2020-09-29 03:23:53,281 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2020-09-29 03:23:53,282 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpm1duseyg
[2020-09-29 03:23:57,772 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-09-29 03:24:04,295 INFO] Summarizer(
  (bert): 

### 3.2. Đánh giá
Chạy đoạn code dưới đây để cài đặt package ROUGE để tính hiệu năng cho mô hình

In [ ]:
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path /content/drive/MyDrive/bai14_home/BertSum/pyrouge/tools/ROUGE-1.5.5
!sudo cpan App::cpanminus
!sudo cpanm XML::DOM
!python pyrouge/setup.py install


os.chdir("/content/drive/MyDrive/bai14_home/BertSum/pyrouge/tools/ROUGE-1.5.5/data/WordNet-2.0-Exceptions")
if os.path.isfile("./WordNet-2.0.exc.db"):
  os.remove("./WordNet-2.0.exc.db")
! ./buildExeptionDB.pl ./ ./smart_common_words.txt ./WordNet-2.0.exc.db

if os.path.isfile("../WordNet-2.0.exc.db"):
  os.remove("../WordNet-2.0.exc.db")

os.chdir("/content/drive/MyDrive/bai14_home/BertSum/pyrouge/tools/ROUGE-1.5.5/data")
!ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

fatal: destination path 'pyrouge' already exists and is not an empty directory.
2020-09-29 05:24:28,255 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/BertSum/pyrouge/tools/ROUGE-1.5.5.
Loading internal null logger. Install Log::Log4perl for logging messages
Reading '/root/.cpan/Metadata'
  Database was generated on Tue, 29 Sep 2020 00:17:02 GMT
App::cpanminus is up to date (1.7044).
XML::DOM is up to date. (1.46)
running install
running bdist_egg
running egg_info
writing pyrouge.egg-info/PKG-INFO
writing dependency_links to pyrouge.egg-info/dependency_links.txt
writing requirements to pyrouge.egg-info/requires.txt
writing top-level names to pyrouge.egg-info/top_level.txt
package init file 'pyrouge/__init__.py' not found (or not a regular file)
writing manifest file 'pyrouge.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pyrouge
c

In [ ]:
os.chdir("/content/drive/MyDrive/bai14_home/BertSum")

In [ ]:
!python src/train.py -mode validate \
-bert_data_path bert_data/ -model_path checkpoints/classifier/ -visible_gpus 0  -gpu_ranks 0 \
-batch_size 30000  -log_file logs/test_results.log -result_path results/ \
-test_all -block_trigram true -bert_config_path bert_config_uncased_base.json -temp_dir temp

[2020-09-29 05:52:28,460 INFO] Loading checkpoint from checkpoints/classifier/model_step_2000.pt
Namespace(accum_count=1, batch_size=30000, bert_config_path='bert_config_uncased_base.json', bert_data_path='bert_data/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='logs/test_results.log', lr=1, max_grad_norm=0, mode='validate', model_path='checkpoints/classifier/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='results/', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='temp', test_all=True, test_from='', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2020-09-29 05:53:04,687 INFO] Loading valid dataset from bert_data/.valid.0.bert.pt, number of examples: 1220
gpu_rank 0
[2020-09-29 05:53:04,695 INFO] * number o

In [ ]:
with open("logs/test_results.log", "r") as f:
  result = f.read()
print(result)

[2020-09-29 02:59:05,901 INFO] PPL []
[2020-09-29 05:24:35,001 INFO] Loading checkpoint from checkpoints/classifier/model_step_2000.pt
[2020-09-29 05:52:28,460 INFO] Loading checkpoint from checkpoints/classifier/model_step_2000.pt
[2020-09-29 05:53:04,687 INFO] Loading valid dataset from bert_data/.valid.0.bert.pt, number of examples: 1220
[2020-09-29 05:53:04,695 INFO] * number of parameters: 109483009
[2020-09-29 05:53:28,624 INFO] Validation xent: 4.91214 at step 2000
[2020-09-29 05:53:28,690 INFO] Loading checkpoint from checkpoints/classifier/model_step_4000.pt
[2020-09-29 05:54:07,141 INFO] Loading valid dataset from bert_data/.valid.0.bert.pt, number of examples: 1220
[2020-09-29 05:54:07,144 INFO] * number of parameters: 109483009
[2020-09-29 05:54:31,077 INFO] Validation xent: 4.84471 at step 4000
[2020-09-29 05:54:31,105 INFO] Loading checkpoint from checkpoints/classifier/model_step_6000.pt
[2020-09-29 05:55:09,693 INFO] Loading valid dataset from bert_data/.valid.0.bert.pt

### 3.3. Huấn luyện mô hình với những loại encoder khác

In [ ]:
!python src/train.py -mode train \
-encoder transformer \
-dropout 0.1 \
-bert_data_path bert_data/ \
-model_path checkpoints/transformer \
-lr 2e-3 -visible_gpus 0  -gpu_ranks 0 \
-report_every 50 -save_checkpoint_steps 100 \
-batch_size 3000 -decay_method noam \
-train_steps 500 -accum_count 2 \
-log_file logs/results.log -use_interval true \
-warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8


[2020-09-29 05:25:18,748 INFO] Device ID 0
[2020-09-29 05:25:18,748 INFO] Device cuda
[2020-09-29 05:25:18,882 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2020-09-29 05:25:18,882 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmppfzwyk84
[2020-09-29 05:25:30,151 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-09-29 05:25:36,754 INFO] Summarizer(
  (bert): 

In [ ]:
!python src/train.py -mode train \
-encoder rnn -dropout 0.1 -bert_data_path bert_data/ \
-model_path checkpoints/rnn -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 \
-report_every 50 -save_checkpoint_steps 100 \
-batch_size 3000 -decay_method noam -train_steps 500 \
-accum_count 2 -log_file logs/results.log \
-use_interval true -warmup_steps 10000 -rnn_size 768 -dropout 0.1


[2020-09-29 05:38:11,170 INFO] Device ID 0
[2020-09-29 05:38:11,170 INFO] Device cuda
[2020-09-29 05:38:11,262 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2020-09-29 05:38:11,263 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpg2x1imwx
[2020-09-29 05:38:15,815 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-09-29 05:38:22,352 INFO] Summarizer(
  (bert): 